In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [3]:
base_dir = "./data/"

In [13]:
def select_sample(data, n=1):
    np.random.shuffle(data)
    negatives = []
    positives = []
    straights = []
    for d in data:
        if d[1] > 0:
            positives.append(d)
        elif d[1] < 0:
            negatives.append(d)
        else:
            straights.append(d)
    return np.array(negatives[:n] + positives[:n] + straights[:n])

def get_model():
    ch, row, col = 3, 160, 320  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/255.0,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse", metrics=['mse'])
    return model
    
def data_gen(data):
    while 1:
        np.random.shuffle(data)
        for d in data:
            img = plt.imread(base_dir + d[0])
            #res = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
            res = np.expand_dims(img, axis=0)
            yield res, np.array([d[1]])
        
def predict_gen(data):
    while 1:
        for d in data:
            img = plt.imread(base_dir + d[0])
            #res = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
            res = np.expand_dims(img, axis=0)
            yield res
            
def read_data(array):
    array_size = array.shape[0]
    X_train = np.zeros((array_size, 160, 320, 3))
    y_train = np.zeros(array_size)
    print(X_train.shape)
    for i, x in enumerate(array):
        img = plt.imread(base_dir + x[0])
        X_train[i,:,:,:] = img
        y_train[i] = x[1]
    return X_train, y_train 

In [14]:
data = pd.read_csv("./data/driving_log.csv")[['center', 'steering']].values
sample = select_sample(data)
X_train, y_train = read_data(sample)

(3, 160, 320, 3)


In [18]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen.fit(X_train)

In [37]:
training_data = sample
epochs = 50
validation_data = sample


model = get_model()
model.fit_generator(datagen.flow(X_train, y_train, batch_size=3), samples_per_epoch=training_data.shape[0],
                    nb_epoch=epochs, validation_data=datagen.flow(X_train, y_train, batch_size=32),
                    nb_val_samples=validation_data.shape[0])

Epoch 1/50
3/3 [==============================] - 0s - loss: 8.9856e-04 - mean_squared_error: 8.9856e-04 - val_loss: 3.1102e-04 - val_mean_squared_error: 3.1102e-04
Epoch 2/50
3/3 [==============================] - 0s - loss: 3.0717e-04 - mean_squared_error: 3.0717e-04 - val_loss: 0.0351 - val_mean_squared_error: 0.0351
Epoch 3/50
3/3 [==============================] - 0s - loss: 0.0368 - mean_squared_error: 0.0368 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 4/50
3/3 [==============================] - 0s - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 1.1370e-04 - val_mean_squared_error: 1.1370e-04
Epoch 5/50
3/3 [==============================] - 0s - loss: 1.1792e-04 - mean_squared_error: 1.1792e-04 - val_loss: 0.0134 - val_mean_squared_error: 0.0134
Epoch 6/50
3/3 [==============================] - 0s - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.0504 - val_mean_squared_error: 0.0504
Epoch 7/50
3/3 [==============================] - 0s - loss: 0.050

In [38]:
model.predict_generator(datagen.flow(X_train, y_train, batch_size=3, shuffle=False), training_data.shape[0])

array([[-0.0486014 ],
       [ 0.00955145],
       [-0.00351881]], dtype=float32)

In [39]:
training_data

array([['IMG/center_2016_12_01_13_33_15_847.jpg', -0.05026283],
       ['IMG/center_2016_12_01_13_43_29_827.jpg', 0.01391724],
       ['IMG/center_2016_12_01_13_35_05_497.jpg', 0.0]], dtype=object)